In [ ]:
import pickle
with open('amewoo_mop_params_48_obs1.0_ent0.0001_dist0.01_pow0.4.pkl', 'rb') as f:
    params = pickle.load(f)
with open('amewoo_mop_losses_48_obs1.0_ent0.0001_dist0.01_pow0.4.pkl', 'rb') as f:
    losses = pickle.load(f)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline

fig, ax = plt.subplots(1, 1, figsize=(8, 6))

# plot the grid search results
ax.plot(losses['total'], label='mixture of products loss')
ax.legend(loc="upper right")
ax.set(
    xlabel=f"training steps",
    ylabel=f"loss value",
    title="mixture of products training: ew=1e-4, dw=1e-2, dp=0.4, n=10"
    
)
plt.show()